# AI Agent - AntiFraud Agent

## Install

In [1]:
!pip -q install google-genai

In [2]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Imports

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata
from google import genai
from IPython.display import HTML, Markdown
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

# Setup

In [5]:
warnings.filterwarnings("ignore")
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# Functions

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Agent 1 - Core-AntiFraud

In [7]:
##########################################
# --- Agente 1: Agente Principal: Classificador de fraudes (CORE-ANTIFRAUD) --- #
##########################################

def agente_core_antifraud(topico):

    core_antifraud = Agent(
        name="agente_core_antifraude",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente especialista em anti fraudes. A sua tarefa é avaliar o tópico e determimnar se há indícios de padrões de golpes e engenharia social sendo aplicada no tópico.
        Após essa análise, gere um relatório sobre o counteúdo e sua conclusão, como golpe provável, suspeito, seguro ou indeterminado.
        Detecte padrões linguísticos típicos de engenharia social (urgência, medo, recompensa fácil, prazo final, você ganhou). Avalie o tom emocional/manipulativo.
        Junto do seu relatório final deve constar um prompt que será enviado para um agente de inteligência artificial que fará buscas sobre golpes similares na internet,
        este prompt final deve ser trabalhdo para que a busca na internet retorne resultados relevantes e com casos similares de golpes,
        anexe ao prompt a mensagem principal da fraude, com textos mais suspeitos, url, links, e-mails, números de telefones, cpf, cnpj, domínios, redes sociais caso sejam informados.
        """,
        description="Agente CORE-ANTIFRAUD, que avalia o texto em busca de padrôes de fraude e engenharia social",
        tools=[google_search]
    )

    entrada_do_agente_core_antifraud = f"Tópico: {topico}"

    lancamentos = call_agent(core_antifraud, entrada_do_agente_core_antifraud)
    return lancamentos

# Agente 2 - Agente especialista em buscas na internet

In [10]:
################################################
# --- Agente 2: Especialista em buscas na internet --- #
################################################

def agente_buscador(topico, lancamentos_buscados):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um especialista em investigação com forte expertise em consultas de golpes e fraudes em geral, você deve:
        usar a ferramenta de busca do Google (google_search) para identificar golpes similares ao proposto no tópico,
        Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar.

        consultandos sites sobre fraudes, golpes, banco de dados de phishing, malwares, serviços como virustotal, google safe browsing,
        abuseipdb e serviços similares. verifique se as url, links, e-mails, números de telefones, cpf, cnpj, domínios, redes sociais, caso sejam informados no tópico, são suspeitos ou maliciosos.
        Aponte falsificações, domínios parecidos, e-mails spoofados. analise fóruns, sites de reclamções, redes sociais.

        Ao final, você irá escolher os temas mais relevante e similares ao tópico com base nas suas pesquisas
        e retornar esses temas, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente. Faça um resumo do tópico com sua análise.
        Este texto final deve conter um prompt que será enviado para um agente de inteligência artificial que fará fará uma análise contextual do seu texto final.
        """,
        description="Agente especialista em busca de internet",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    busca_result = call_agent(buscador, entrada_do_agente_buscador)
    return busca_result

# Agente 3 - Agente Contextual

In [9]:
################################################
# --- Agente 2: Contextual --- #
################################################

def agente_contextual(topico, busca_result):
    contextual = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um especialista em investigação com forte expertise em consultas de golpes e fraudes em geral,
        o tópico em questão é resultado de um trabalho de investigação de um agente de inteligência artificial especialista em investigação de golpes e fraudes,
        você deve:
        analisar o tópico em busca de padrões de fraudes e golpes, comparando o conteúdo com padrões legais ou práticas legítimas:

        ####

        exemplos:
        Ofertas de trabalhos não solicitadas com salários irreais.
        Pedidos de pagamentos antecipados por "taxasa administrativas".
        Empréstimos que exigem depósito prévio.

        Analise o tópico e verifique se o mesmo se encaixa em uma tentativa de golpe e faça uma conclusão se o mesmo é um golpe provável, suspeito, seguro ou indeterminado.
        """,
        description="Agente especialista em busca de internet",
        tools=[google_search]
    )

    entrada_do_agente_contextual = f"Tópico:{topico}\nResultados buscados: {busca_result}"
    # Executa o agente
    final_result = call_agent(contextual, entrada_do_agente_contextual)
    return final_result

In [17]:
print("🚀 Iniciando o Sistema de Detecção de golpes e fraudes com 3 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer verificar golpes e fraudes: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos verificar se a mensagem informada se trata de um fraude ou golpe. {topico}")

    result_agent_core_antifraud = agente_core_antifraud(topico)
    print("\n--- 📝 Resultado do Agente 1 (core_antifraud) ---\n")
    display(to_markdown(result_agent_core_antifraud))
    print("--------------------------------------------------------------\n\n")

    result_agent_buscador = agente_buscador(topico, result_agent_core_antifraud)
    print("\n--- 📝 Resultado do Agente 2 (buscador) ---\n")
    display(to_markdown(result_agent_buscador))
    print("--------------------------------------------------------------\n\n")

    result_agent_contextual = agente_contextual(topico, result_agent_buscador)
    print("\n--- 📝 Resultado do Agente 3 (Contextual) ---\n")
    display(to_markdown(result_agent_contextual))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Detecção de golpes e fraudes com 3 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer verificar golpes e fraudes: recebi a seguinte mensagem e gostaria de saber se é um golpe:  ### mensagem:  Bradescard: Converta sus 66.261 pontos em benefícios exclusivos! Aproveite itens, viagens e abatimentos na fatura. Acesse o link liberadormilhas.com/7h
Maravilha! Vamos verificar se a mensagem informada se trata de um fraude ou golpe. recebi a seguinte mensagem e gostaria de saber se é um golpe:  ### mensagem:  Bradescard: Converta sus 66.261 pontos em benefícios exclusivos! Aproveite itens, viagens e abatimentos na fatura. Acesse o link liberadormilhas.com/7h

--- 📝 Resultado do Agente 1 (core_antifraud) ---



> Aqui está uma avaliação da mensagem que você recebeu, focada na identificação de possíveis elementos de golpe ou engenharia social:
> 
> **Análise:**
> 
> *   **Urgência e Recompensa:** A mensagem tenta criar um senso de urgência ("Aproveite") combinada com a promessa de benefícios exclusivos. Essa é uma tática comum para atrair a atenção e levar a pessoa a agir impulsivamente.
> *   **Valor Específico:** O valor exato de pontos (66.261) pode parecer específico e, portanto, mais legítimo. No entanto, golpistas frequentemente usam números aleatórios para dar credibilidade.
> *   **Link Suspeito:** O domínio "liberadormilhas.com" não parece ser um domínio oficial do Bradescard. Golpistas frequentemente usam URLs ligeiramente diferentes dos originais para enganar as pessoas (phishing).
> *   **Genérico:** A mensagem é genérica, poderia ser enviada para qualquer pessoa que tenha um cartão Bradescard.
> 
> **Relatório:**
> 
> *   **Conclusão:** Suspeito. Há vários indicadores que sugerem que esta mensagem pode ser uma tentativa de golpe (phishing). O link suspeito e a combinação de urgência com recompensa são sinais de alerta.
> *   **Padrões de Engenharia Social:** A mensagem explora o desejo de obter vantagens (milhas, viagens) e a possível ansiedade de perder uma oportunidade.
> 
> **Prompt para Agente de Inteligência Artificial:**
> 
> Para auxiliar na identificação de golpes similares, sugiro o seguinte prompt para um agente de inteligência artificial especializado em buscas na internet:
> 
> 
> ```
> Identifique e liste casos de golpes e phishing que utilizam mensagens SMS ou e-mail com as seguintes características:
> 
> *   Remetente se passando pelo Bradescard ou outras instituições financeiras.
> *   Promessa de conversão de pontos de programas de fidelidade em benefícios (viagens, descontos).
> *   Utilização de links encurtados ou domínios que não correspondem ao site oficial da instituição.
> *   Foco em criar um senso de urgência para que o usuário clique no link e forneça dados pessoais ou financeiros.
> *   Mencione o domínio "liberadormilhas.com".
> 
> Inclua na busca os seguintes termos e informações da mensagem original:
> 
> *   "Bradescard"
> *   "Converter pontos em benefícios"
> *   "liberadormilhas.com"
> *   "66.261 pontos"
> *   "SMS"
> *   "Phishing"
> ```
> 
> 
> Recomendações Adicionais:
> 
> *   **Não clique no link:** Se você tiver qualquer dúvida sobre a legitimidade da mensagem, não clique no link.
> *   **Verifique Diretamente:** Entre em contato com o Bradescard através dos canais oficiais (site, aplicativo, telefone) para verificar se a mensagem é verdadeira.
> *   **Denuncie:** Se você suspeitar de um golpe, denuncie às autoridades competentes e ao Bradescard.
> 


--------------------------------------------------------------



--- 📝 Resultado do Agente 2 (buscador) ---



> Com base na análise fornecida, a mensagem recebida tem características que a classificam como suspeita de ser um golpe. Para aprofundar a investigação, realizei as seguintes buscas:
> 
> 
> ### Resumo da Análise:
> 
> A mensagem recebida se encaixa em um padrão de golpes e phishing que utilizam o nome de instituições financeiras como o Bradescard, com o objetivo de roubar dados pessoais e financeiros dos usuários. As principais características desse tipo de golpe incluem:
> 
> *   **Remetente:** Se passando pelo Bradescard ou outra instituição financeira.
> *   **Isca:** Promessa de conversão de pontos de programas de fidelidade em benefícios (viagens, descontos, dinheiro).
> *   **Meio:** Utilização de mensagens SMS ou e-mail.
> *   **Link Malicioso:** Utilização de links encurtados ou domínios que não correspondem ao site oficial da instituição (ex: liberadormilhas.com).
> *   **Urgência:** Foco em criar um senso de urgência para que o usuário clique no link e forneça dados rapidamente.
> 
> A análise das buscas revelou diversos casos semelhantes envolvendo o Bradesco e outros bancos, nos quais os golpistas utilizam a mesma estratégia de oferecer vantagens em troca de informações pessoais. O domínio "liberadormilhas.com" é um forte indicador de fraude.
> 
> ### Temas Relevantes:
> 
> 1.  **Golpes de Phishing com Milhas/Pontos:** Análise detalhada de como esses golpes funcionam, exemplos de mensagens e sites falsos.
> 2.  **Smishing (Phishing por SMS):** Como identificar e se proteger de golpes que chegam por SMS, com foco em mensagens que usam nomes de bancos e programas de fidelidade.
> 3.  **Domínios Suspeitos e Encurtadores de URL:** Como verificar a legitimidade de um domínio e os riscos de clicar em links encurtados.
> 4.  **Engenharia Social:** As táticas psicológicas utilizadas pelos golpistas para manipular as vítimas (urgência, medo de perder uma oportunidade).
> 5.  **Alertas e Prevenção:** O que o Bradesco e outras instituições financeiras recomendam para evitar cair nesses golpes.
> 6.  **O que fazer se cair no golpe:** Passos a seguir caso você tenha fornecido seus dados em um site falso.
> 
> ### Plano para o Post:
> 
> 1.  **Introdução:** Explicar o golpe recebido e por que ele é suspeito (urgência, link estranho, etc.).
> 2.  **O que é Phishing e Smishing:** Definir os termos e explicar como esses golpes funcionam.
> 3.  **Exemplos de Golpes com Milhas/Pontos:** Mostrar exemplos de mensagens e sites falsos que utilizam essa isca.
> 4.  **Como Identificar um Domínio Suspeito:** Explicar como verificar a legitimidade de um domínio e os riscos de clicar em links encurtados.
> 5.  **Táticas de Engenharia Social:** Detalhar as táticas psicológicas utilizadas pelos golpistas (urgência, medo de perder uma oportunidade) e como elas funcionam.
> 6.  **Recomendações de Segurança do Bradesco:** Listar as dicas de segurança fornecidas pelo Bradesco e outras instituições financeiras.
> 7.  **O que Fazer se Cair no Golpe:** Explicar os passos a seguir caso você tenha fornecido seus dados em um site falso (notificar o banco, trocar senhas, registrar um boletim de ocorrência).
> 8.  **Como Denunciar:** Informar como denunciar o golpe às autoridades competentes e ao Bradesco.
> 9.  **Conclusão:** Reforçar a importância de estar sempre atento e desconfiar de mensagens que oferecem vantagens em troca de informações pessoais.
> 
> ### Prompt para Agente de Inteligência Artificial:
> 
> 
> ```
> Analise o seguinte texto sobre um possível golpe envolvendo o Bradescard e um link suspeito (liberadormilhas.com):
> 
> [Texto da análise acima]
> 
> Com base nessa análise, gere um conteúdo informativo e educativo sobre como identificar e se proteger de golpes de phishing e smishing que utilizam o nome de instituições financeiras e programas de fidelidade. O conteúdo deve incluir:
> 
> *   Uma explicação detalhada de como esses golpes funcionam, com exemplos de mensagens e sites falsos.
> *   Dicas práticas para verificar a legitimidade de um domínio e identificar links suspeitos.
> *   Uma análise das táticas de engenharia social utilizadas pelos golpistas para manipular as vítimas.
> *   Recomendações de segurança do Bradesco e outras instituições financeiras.
> *   Instruções sobre o que fazer se você cair no golpe, incluindo como notificar o banco, trocar senhas e registrar um boletim de ocorrência.
> *   Informações sobre como denunciar o golpe às autoridades competentes.
> 
> O objetivo é criar um guia completo e fácil de entender que ajude os usuários a se protegerem contra esses tipos de fraude.
> ```


--------------------------------------------------------------



--- 📝 Resultado do Agente 3 (Contextual) ---



> Com base na análise fornecida, a mensagem que você recebeu é **altamente suspeita** de ser um golpe. As características da mensagem (uso do nome Bradescard, promessa de benefícios por pontos, link suspeito) se alinham com padrões de phishing e smishing já identificados.
> 
> Para sua segurança, **não clique no link** `liberadormilhas.com/7h` e **não forneça nenhuma informação pessoal** que possa ser solicitada em uma página que se abra a partir desse link.
> 
> A seguir, um resumo dos pontos que indicam ser um golpe e o que fazer para se proteger:
> 
> *   **Domínio suspeito:** O link `liberadormilhas.com` não parece ser um domínio oficial do Bradescard. Domínios oficiais de bancos geralmente incluem o nome do banco de forma clara e não utilizam encurtadores de URL.
> *   **Urgência:** Golpistas frequentemente criam um senso de urgência para que você aja impulsivamente, sem verificar a legitimidade da oferta.
> *   **Phishing/Smishing:** Este é um golpe clássico de phishing (se enviado por e-mail) ou smishing (se enviado por SMS), onde criminosos se passam por instituições legítimas para roubar seus dados.
> 
> **O que fazer para se proteger:**
> 
> 1.  **Desconfie de ofertas:** Sempre desconfie de mensagens não solicitadas que oferecem vantagens ou benefícios em troca de informações pessoais.
> 2.  **Verifique o remetente:** Verifique se o remetente da mensagem é um endereço de e-mail ou número de telefone oficial do Bradescard.
> 3.  **Não clique em links:** Nunca clique em links suspeitos. Em vez disso, acesse o site oficial do Bradescard digitando o endereço diretamente no seu navegador.
> 4.  **Contate o Bradescard:** Em caso de dúvida, entre em contato com o Bradescard por meio dos canais de atendimento oficiais (telefone, site) para verificar a veracidade da mensagem.
> 5.  **Proteja seus dados:** Nunca forneça informações pessoais (senhas, números de cartão de crédito, etc.) em sites ou formulários suspeitos.
> 6.  **Mantenha seu software atualizado:** Mantenha seu sistema operacional, navegador e antivírus sempre atualizados para se proteger contra malware e outras ameaças.
> 
> **Se você já clicou no link ou forneceu dados:**
> 
> 1.  **Contate o Bradescard imediatamente:** Informe o banco sobre o ocorrido e siga as orientações fornecidas.
> 2.  **Troque suas senhas:** Altere as senhas de todas as suas contas online, especialmente as contas bancárias e de e-mail.
> 3.  **Monitore suas contas:** Fique atento a qualquer atividade suspeita em suas contas bancárias e de cartão de crédito.
> 4.  **Registre um boletim de ocorrência:** Denuncie o golpe à polícia, registrando um boletim de ocorrência.
> 
> Em resumo, a mensagem que você recebeu é **provavelmente um golpe**. Siga as dicas acima para se proteger e evitar ser vítima de fraudes.
> 


--------------------------------------------------------------


# Exemplos de golpes

Exemplo 1:

recebi a seguinte mensagem e gostaria de saber se é um golpe:

###
mensagem:

Olá você acabou de ganhar um prêmio no valor de R$ 3.500,00. Par retirar o seu prêmio envie um pix de R$ 100,00 para a cahve pix 12345678988, este valor se refera a taxa de imposto de renda.

exemplo 2:

recebi a seguinte mensagem e gostaria de saber se é um golpe:

###
mensagem:

Olá aqui é da floricultura "Flor do Amor", estamos enviando um bouquet de rosas de cortesia, para recebê-lo basta pagar uma taxa de frete de R$ 5,00 ao motoboy, essa taxa deve ser paga via cartão de crédito.

exemplos 3:

recebi a seguinte mensagem e gostaria de saber se é um golpe:

###
mensagem:

Olá aqui é do banco do brasil, estamos ligando para confirmar se você fez uma compra no valor de R$ 5000,00. Caso não tenha sido você digite 2 para falar direto com nossa central de atendimento, pois essa compra parece suspeita e o seu cartão será bloqueado. Caso tenha sido você, desconsidere esta mensagem.


exemplos 4

recebi a seguinte mensagem e gostaria de saber se é um golpe:

###
mensagem:

Bradescard: Converta sus 66.261 pontos em benefícios exclusivos! Aproveite itens, viagens e abatimentos na fatura.
Acesse o link liberadormilhas.com/7h